In [1]:
path = "/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/merged_data_dated.csv"

In [2]:
import pandas as pd

In [5]:
df = pd.read_csv(path)
df = df[df.apply(lambda x: "pull" not in str(x.issue_url), axis=1)]
# df = df.dropna()
# df["issue_title"] = df.issue_title.to_string()

titles = df.issue_title.to_list()

In [6]:
len(titles)

7578

In [5]:
# titles = titles[:10000]

In [8]:
import gensim
from gensim import corpora
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [9]:
stop_words = set(stopwords.words('english'))
punctuations = set(string.punctuation)
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words and token not in punctuations]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(tokens)

corpus = titles # [preprocess(doc) for doc in titles]

In [10]:
from torch import cuda

model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

cuda:0


In [11]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [12]:
# Llama 2 Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [13]:
# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

In [14]:
prompt = "Could you explain to me how 4-bit quantization works as if I am 5?"
res = generator(prompt)
print(res[0]["generated_text"])

Could you explain to me how 4-bit quantization works as if I am 5?
 everybody knows that 4 bits is not enough for most things, but it's a good starting point.

For example, let's say we want to represent the number 10 in base 10. We could write it as 4 digits: 1-0-0-0. But wait! We can do better than that! We can use just 2 digits: 1-0. That's right! With only 2 digits, we can still represent all numbers from 0 to 9! And that's exactly what 4-bit quantization does. It takes a big number and breaks it down into smaller parts so that we can represent it with fewer bits.

So, when we have a big number like 10, we break it down into 4 smaller parts: 1-0-0-0. Then we take each part and assign it a special number called a "quantizer." The quantizer helps us decide which of the 4 parts to keep and which to throw away. For example, if we want to keep the first part (1), we might throw away the second part (0). Why? Because the first part tells us that there are 10 ones in the original number, 

In [15]:
# System prompt describes information given to all conversations
system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""

In [16]:
# Example prompt demonstrating the output we are looking for
example_prompt = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.

[/INST] Environmental impacts of eating meat
"""

In [17]:
# Our main prompt with documents ([DOCUMENTS]) and keywords ([KEYWORDS]) tags
main_prompt = """
[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.
[/INST]
"""

In [18]:
prompt = system_prompt + example_prompt + main_prompt

In [19]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("BAAI/bge-small-en")
embeddings = embedding_model.encode(corpus, show_progress_bar=True)

Batches:   0%|          | 0/237 [00:00<?, ?it/s]

In [22]:
from umap import UMAP
from hdbscan import HDBSCAN

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [23]:
# Pre-reduce embeddings for visualization purposes
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [24]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# Text generation with Llama 2
llama2 = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "Llama2": llama2,
    "MMR": mmr,
}

In [25]:
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=20)

In [26]:
from bertopic import BERTopic
# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer_model = CountVectorizer(stop_words="english")

topic_model = BERTopic(

  # Sub-models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=cluster_model,
  representation_model=representation_model,
  # vectorizer_model=vectorizer_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True,
  nr_topics=50
)

# Train model
topics, probs = topic_model.fit_transform(corpus, embeddings)

2024-03-07 22:04:00,258 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-07 22:04:20,306 - BERTopic - Dimensionality - Completed ✓
2024-03-07 22:04:20,308 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-07 22:04:20,734 - BERTopic - Cluster - Completed ✓
2024-03-07 22:04:20,735 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 20/20 [00:09<00:00,  2.21it/s]
2024-03-07 22:04:30,827 - BERTopic - Representation - Completed ✓
2024-03-07 22:04:30,829 - BERTopic - Topic reduction - Reducing number of topics
2024-03-07 22:04:30,831 - BERTopic - Topic reduction - Reduced number of topics from 20 to 20


In [27]:
# Show topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,Llama2,MMR,Representative_Docs
0,0,26,0_hcrlateattachworkload_0_hcrlateattachworkloa...,"[hcrlateattachworkload_0, hcrlateattachworkloa...","[hcrlateattachworkload_0_failed, hcrlateattach...","[Java Crashes and HCRLateAttachWorkload, , , ,...","[hcrlateattachworkload_0, hcrlateattachworkloa...",[extended.system JIT crash running HCRLateAtta...
1,1,516,1_gc_cache_for_of,"[gc, cache, for, of, shared, on, balanced, in,...","[aarch64, aarch32, benchmark, jit, throughput,...","[Garbage Collection (GC) Support, , , , , , , ...","[gc, cache, for, of, shared, on, balanced, in,...",[ConstantDynamic: Add support for balanced GC ...
2,2,489,2_java_openjdk_outofmemoryerror_jdk11,"[java, openjdk, outofmemoryerror, jdk11, lang,...","[jdk17, openjdk, jdk8, jdk15, jdk, jdk18, jdk1...","[Java Heap Space Errors, , , , , , , , , ]","[java, openjdk, outofmemoryerror, jdk11, lang,...",[jdk17 acceptance OpenJDK java/lang/System/Log...
3,3,479,3_jenkins_build_to_builds,"[jenkins, build, to, builds, docker, pr, on, f...","[jenkins, openj9, buildenv, docker, dockerfile...","[Jenkins automation, , , , , , , , , ]","[jenkins, build, to, builds, docker, pr, on, f...","[Add functional test in OpenJ9 Jenkins, Add Je..."
4,4,564,4_openj9_hotspot_to_with,"[openj9, hotspot, to, with, build, on, j9, for...","[openj9, openjdk9, openjdk8, openjdk, adoptope...","[OpenJ9 build issues, , , , , , , , , ]","[openj9, hotspot, to, with, build, on, j9, for...",[Assertion failed : Eclipse OpenJ9 VM AdoptOpe...
5,5,245,5_jtreg_java_test_failure,"[jtreg, java, test, failure, lang, fail, invok...","[jtreg, genericstringtest, java, jdk, jdk11, d...","[Java testing failures, , , , , , , , , ]","[jtreg, java, test, failure, lang, fail, invok...",[JTReg test fail : java/lang/invoke/ProtectedM...
6,6,603,6_functional_test_jdk11_64_cmprssptrs,"[functional, test, jdk11, 64_cmprssptrs, faile...","[cmdlinetester_jep178_staticlinking_se80_0, cm...","[Java Functional Testing Issues, , , , , , , ,...","[functional, test, jdk11, 64_cmprssptrs, faile...",[Test-extended.functional-JDK11-linux_x86-64_c...
7,7,338,7_hang_test_mauve_jdk8,"[hang, test, mauve, jdk8, failed, system, test...","[jdk8, lambdaloadtest_concurrentscavenge_0, te...","[Java testing issues, , , , , , , , , ]","[hang, test, mauve, jdk8, failed, system, test...",[Test-extended.system-JDK8-linux_x86-64_cmprss...
8,8,365,8_jvm_jdmpview_javacore_attach,"[jvm, jdmpview, javacore, attach, core, dump, ...","[javacore, openj9, jdmpview, openj9_java_optio...","[JVM Core Dump Issues, , , , , , , , , ]","[jvm, jdmpview, javacore, attach, core, dump, ...",[Jdmpview from 11.0.11+2-202102111813 does not...
9,9,386,9_remove_for_the_test,"[remove, for, the, test, to, tests, code, in, ...","[deprecate, exclude, deprecated, remove, code,...","[Code cleanup and optimization, , , , , , , , , ]","[remove, for, the, test, to, tests, code, in, ...",[Remove deprecated methods from java.lang.Acce...


In [28]:
topic_model.get_topics()

{0: [('hcrlateattachworkload_0', 0.3058745329747455),
  ('hcrlateattachworkload', 0.15758606025729582),
  ('hcrlateattachworkload_previewenabled', 0.15758606025729582),
  ('crash', 0.14932929074415435),
  ('major', 0.13369595466235995),
  ('hcrlateattachworkload_previewenabled_0', 0.12329062974946942),
  ('unsupported', 0.117373324723258),
  ('vmstate', 0.11616854603632154),
  ('system', 0.10470969208611358),
  ('extended', 0.09429996887939718)],
 1: [('gc', 0.06253783828958669),
  ('cache', 0.04018562261538806),
  ('for', 0.03898644942693011),
  ('of', 0.03471830174223314),
  ('shared', 0.033292909747156846),
  ('on', 0.032365611463621526),
  ('balanced', 0.03206924424544293),
  ('in', 0.03174173198582588),
  ('support', 0.027766850919997903),
  ('heap', 0.025771989415424565)],
 2: [('java', 0.06824462262965854),
  ('openjdk', 0.03884993499979732),
  ('outofmemoryerror', 0.03142265512170632),
  ('jdk11', 0.029260047915056816),
  ('lang', 0.028477588305072733),
  ('jdk17', 0.0282102399

In [29]:
topic_model.get_topic(1, full=True)["KeyBERT"]

[('aarch64', 0.90850145),
 ('aarch32', 0.8978165),
 ('benchmark', 0.8957604),
 ('jit', 0.8926149),
 ('throughput', 0.8917089),
 ('balanced', 0.88678336),
 ('implementation', 0.88563377),
 ('stack', 0.88276935),
 ('gc', 0.8822202),
 ('allocation', 0.8814466)]

In [30]:
llama2_labels = [label[0][0].split("\n")[0] for label in topic_model.get_topics(full=True)["Llama2"].values()]
topic_model.set_topic_labels(llama2_labels)

In [35]:
topic_model.visualize_documents(titles, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True, title="OpenJ9 Issue Topics")

In [32]:
df["topic_id"] = topics

In [39]:
df.head()

,Unnamed: 0,issue_number,issue_url,issue_title,issue_body,issue_state,creator,comments,assignees,labels,created_at,closed_at,topic_id
1,41,2,https://github.com/eclipse-openj9/openj9/issues/2,Build instructions link in the README.md point...,The `Build instructions` link in the `README.m...,closed,aarongraham9,NaN,NaN,NaN,2017-09-13T15:47:13Z,2017-09-13T15:58:31Z,3
2,40,3,https://github.com/eclipse-openj9/openj9/issues/3,FAQ link in the README is broken,FAQ link in the README leads to: http://www.ec...,closed,dorrab,NaN,NaN,NaN,2017-09-13T15:50:19Z,2017-09-13T16:09:27Z,3
4,38,5,https://github.com/eclipse-openj9/openj9/issues/5,Link to DockerFile on build instruction page i...,Link for DockerFile on [build instruction page...,closed,r30shah,<comment><user>mstoodle</user><body>Because it...,NaN,NaN,2017-09-13T16:08:10Z,2017-09-13T16:23:07Z,3
10,32,11,https://github.com/eclipse-openj9/openj9/issue...,HOWTO Request: Managing changes across depende...,"Like all projects, OpenJ9 builds on the should...",open,mgaudet,<comment><user>pshipton</user><body>Please cle...,NaN,question,2017-09-14T13:08:31Z,NaN,3
12,30,13,https://github.com/eclipse-openj9/openj9/issue...,Compilation Output is too Verbose,The output when compiling the OpenJ9 source co...,closed,rservant,<comment><user>rwy7</user><body>When openj9 sw...,NaN,"enhancement, comp:build",2017-09-14T14:19:36Z,2018-01-11T21:26:21Z,1


In [38]:
len(df[df["topic_id"] == 15]["assignees"].unique())

38

In [44]:
df["topic_label"] = df["topic_id"].apply(lambda x: topic_model.get_topics(full=True)["Llama2"][x][0][0])

In [46]:
df.head()

,Unnamed: 0,issue_number,issue_url,issue_title,issue_body,issue_state,creator,comments,assignees,labels,created_at,closed_at,topic_id,topic_label
1,41,2,https://github.com/eclipse-openj9/openj9/issues/2,Build instructions link in the README.md point...,The `Build instructions` link in the `README.m...,closed,aarongraham9,NaN,NaN,NaN,2017-09-13T15:47:13Z,2017-09-13T15:58:31Z,3,Jenkins automation
2,40,3,https://github.com/eclipse-openj9/openj9/issues/3,FAQ link in the README is broken,FAQ link in the README leads to: http://www.ec...,closed,dorrab,NaN,NaN,NaN,2017-09-13T15:50:19Z,2017-09-13T16:09:27Z,3,Jenkins automation
4,38,5,https://github.com/eclipse-openj9/openj9/issues/5,Link to DockerFile on build instruction page i...,Link for DockerFile on [build instruction page...,closed,r30shah,<comment><user>mstoodle</user><body>Because it...,NaN,NaN,2017-09-13T16:08:10Z,2017-09-13T16:23:07Z,3,Jenkins automation
10,32,11,https://github.com/eclipse-openj9/openj9/issue...,HOWTO Request: Managing changes across depende...,"Like all projects, OpenJ9 builds on the should...",open,mgaudet,<comment><user>pshipton</user><body>Please cle...,NaN,question,2017-09-14T13:08:31Z,NaN,3,Jenkins automation
12,30,13,https://github.com/eclipse-openj9/openj9/issue...,Compilation Output is too Verbose,The output when compiling the OpenJ9 source co...,closed,rservant,<comment><user>rwy7</user><body>When openj9 sw...,NaN,"enhancement, comp:build",2017-09-14T14:19:36Z,2018-01-11T21:26:21Z,1,Garbage Collection (GC) Support


In [45]:
df.to_csv("/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/openj9_topic_kmeans.csv")

In [40]:
topic_model.visualize_topics()

In [41]:
topic_model.visualize_barchart()

In [36]:
topic_model.save("topic_model_gc", serialization="safetensors")

In [37]:
topic_model.transform(titles[2])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-02-13 20:19:58,601 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.


/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

2024-02-13 20:20:15,347 - BERTopic - Dimensionality - Completed ✓
2024-02-13 20:20:15,349 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2024-02-13 20:20:15,351 - BERTopic - Cluster - Completed ✓


([0], None)

In [38]:
df.head()

,owner,issue_title,description,topic_id,topic_label
0,amit@chromium.org,"Scrolling with some scroll mice (touchpad, etc...",Product Version : <see about:version>URLs...,18,Touch and Scroll Issues
1,jon@chromium.org,Proxy causes some or all network requests to fail,Product Version : 0.2.149.27 (1583)URLs (...,5,Security and SSL issues
2,pfeldman@chromium.org,"Web inspector button ""dock to main window"" doe...",Product Version : chrome beta 1URLs (if a...,0,Webpage rendering regression issues
3,jon@chromium.org,Habari admin interface is not rendered correctly,Product Version : 0.2.149.27 (1583)URLs (...,4,Chrome Tab and Window Behavior Issues
4,pkasting@chromium.org,Maximize on second larger monitor not working,Product Version : 0.2.149.27URLs (if appl...,18,Touch and Scroll Issues


In [39]:
topic_model.visualize_hierarchy()

In [42]:
topic_model.visualize_heatmap()